# Линейная регрессия. Работа с признаками

## Описание задачи и загрузка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

В этом задании мы рассмотрим различные аспекты построения линейной модели. Мы будем работать с одним из классических наборов данных в статистике, содержащим информацию о бриллиантах. Описание можно посмотреть [здесь](https://www.kaggle.com/shivam2503/diamonds).

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/diamonds.csv')
data.head(5)

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


Посмотрим на типы столбцов.

In [3]:
data.dtypes

Unnamed: 0      int64
carat         float64
cut            object
color          object
clarity        object
depth         float64
table         float64
price           int64
x             float64
y             float64
z             float64
dtype: object

Мы будем решать задачу предсказания цены бриллианта `price` в зависимости от его характеристик.

## Построение модели

### Задание 1

Есть ли в наборе данных пропущенные значения? Если да, удалите их.

Также выведите на экран число пропусков в каждом столбце.

In [11]:
missing_values = data.isnull().sum()
missing_values

Unnamed: 0    0
carat         0
cut           0
color         0
clarity       0
depth         0
table         0
price         0
x             0
y             0
z             0
dtype: int64

### Задача 2

Есть ли в наборе данных бессмысленные столбцы (признаки, не несущие дополнительной информации)?  
Если да, то удалите их.

In [14]:
data.head(5)

df_cleaned = data.drop(columns=['Unnamed: 0']) if 'Unnamed: 0' in data.columns else data

df_cleaned.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### Задание 3

Линейная регрессия основана на предположении о линейной связи между признаками и целевой переменной, а потому перед выбором переменных для включения в модель имеет смысл проверить, насколько эта связь выполняется. Для следующих пунктов нам также потребуются корреляции между признаками.  
Выведите матрицу корреляций между всеми вещественными признаками и целевой переменной.

Какой вещественный признак коррелирует с целевой переменной больше всего?

In [18]:
df_cleaned.corr().style.background_gradient(cmap='coolwarm')

<ipython-input-18-93c0ea2a07d3>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_cleaned.corr().style.background_gradient(cmap='coolwarm')


,carat,depth,table,price,x,y,z
carat,1.000000,0.028224,0.181618,0.921591,0.975094,0.951722,0.953387
depth,0.028224,1.000000,-0.295779,-0.010647,-0.025289,-0.029341,0.094924
table,0.181618,-0.295779,1.000000,0.127134,0.195344,0.183760,0.150929
price,0.921591,-0.010647,0.127134,1.000000,0.884435,0.865421,0.861249
x,0.975094,-0.025289,0.195344,0.884435,1.000000,0.974701,0.970772
y,0.951722,-0.029341,0.183760,0.865421,0.974701,1.000000,0.952006
z,0.953387,0.094924,0.150929,0.861249,0.970772,0.952006,1.000000


In [19]:
most_correlated_feature = df_cleaned.corr()['price'].drop('price').idxmax()
most_correlated_value = df_cleaned.corr()['price'][most_correlated_feature]
most_correlated_feature, most_correlated_value

<ipython-input-19-7911820716f2>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  most_correlated_feature = df_cleaned.corr()['price'].drop('price').idxmax()
<ipython-input-19-7911820716f2>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  most_correlated_value = df_cleaned.corr()['price'][most_correlated_feature]


('carat', 0.9215913011934687)

### Задание 4

Так как линейная модель складывает значения признаков с некоторыми весами, нам нужно аккуратно обработать категориальные признаки. Закодируйте категориальные переменные при помощи OneHot-кодирования ([`pd.get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)). Не забудьте поставить значение параметра `drop_first` равным `True`.

Сколько получилось столбцов в таблице `data`?

*P.S. Числовые столбцы оставляем в таблице без изменений.*

In [26]:
df_cleaned.head(5)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [34]:
pd.get_dummies(df_cleaned, columns=['cut', 'color', 'clarity'],drop_first=True)

,carat,depth,table,price,x,y,z,cut_Good,cut_Ideal,cut_Premium,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,1,...,0,1,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,335,4.34,4.35,2.75,1,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50,0,1,0,...,0,0,0,0,1,0,0,0,0,0
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61,1,0,0,...,0,0,0,0,1,0,0,0,0,0
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56,0,0,0,...,0,0,0,0,1,0,0,0,0,0
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74,0,0,1,...,1,0,0,0,0,1,0,0,0,0


In [36]:
df_encoded = pd.get_dummies(df_cleaned, columns=['cut', 'color', 'clarity'], drop_first=True)
print(df_encoded.shape[1])

24


### Задание 5

Создайте матрицу `X`, содержащую все признаки, и не содержащую целевую переменную `price`. Также создайте вектор `y`, содержащий целевую переменную `price`.

In [42]:
# Создание матрицы X, содержащей все признаки, кроме целевой переменной 'price'
X = df_encoded.drop('price', axis=1)

# Создание вектора y, содержащего целевую переменную 'price'
y = df_encoded['price']

X.shape, y.shape

((53940, 23), (53940,))

Разделите выборку на тренировочную и тестовую. Долю тестовой выборки укажите равной `0.3`.

При разбиении укажите `random_state = 42`.

In [43]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state = 42)

### Задание 6

Зачастую при использовании линейных моделей вещественные признаки масштабируются.  В этой задаче масштабируйте вещественные признаки тренировочной и тестовой выборок при помощи модуля `StandardScaler`.

*  Обучите (`fit`) scaler на тренировочных данных
*  Преобразуйте (`transform`) и трейн, и тест

После применения масштабирования матрица перестает быть объектом `pandas.DataFrame` - решите эту проблему.

In [69]:
from sklearn.preprocessing import StandardScaler

# Создание экземпляра StandardScaler
scaler = StandardScaler()

numeric_columns = Xtrain.select_dtypes(include=['float64', 'int64']).columns

# Обучение scaler на тренировочных данных (только на вещественных признаках)
scaler.fit(Xtrain[numeric_columns])

# Преобразование тренировочных и тестовых данных
Xtrain_scaled = Xtrain.copy()
Xtest_scaled = Xtest.copy()

Xtrain_scaled[numeric_columns] = scaler.transform(Xtrain[numeric_columns])
Xtest_scaled[numeric_columns] = scaler.transform(Xtest[numeric_columns])

# Проверка, что результаты остались в формате DataFrame
is_dataframe_train = isinstance(Xtrain_scaled, pd.DataFrame)
is_dataframe_test = isinstance(Xtest_scaled, pd.DataFrame)


print(Xtrain_scaled.shape, Xtest_scaled.shape, is_dataframe_train, is_dataframe_test)
Xtrain_scaled.head()

(37758, 23) (16182, 23) True True


,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
19497,0.862659,-0.311437,-0.207099,1.055581,0.986556,0.968253,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
31229,-1.029889,0.178549,-0.656213,-1.207734,-1.202544,-1.168276,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
22311,0.862659,0.458541,-0.207099,0.904099,0.951670,0.982309,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
278,0.021527,0.598537,-1.105327,0.164512,0.192898,0.251391,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6646,-0.020530,-0.031445,-0.656213,0.182333,0.184176,0.181110,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1


### Задание 7

Обучите линейную регрессию на тренировочной выборке. Выведите *r2-score* на тренировочной и тестовой выборках.

In [70]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model = LinearRegression()

model.fit(Xtrain_scaled, ytrain)

train_pred = model.predict(Xtrain_scaled)
test_pred = model.predict(Xtest_scaled)

r2_score_train = r2_score(ytrain, train_pred)
r2_score_test = r2_score(ytest, test_pred)
print(r2_score_train, r2_score_test)

0.9195976267987521 0.9201866914388087


### Задание 8

Выведите на экран веса, которые линейная регрессия присвоила признакам.

Какой признак имеет наибольший отрицательный вес? (наибольший по модулю среди всех отрицательных весов)

In [71]:
# Получение весов (коэффициентов) линейной регрессии
feature_weights = model.coef_

# Сопоставление весов с соответствующими признаками
feature_weights_dict = dict(zip(Xtrain_scaled.columns, feature_weights))

# Вывод весов
print(feature_weights_dict)

# Определение признака с наибольшим отрицательным весом
most_negative_feature = min(feature_weights_dict, key=lambda k: feature_weights_dict[k])
most_negative_weight = feature_weights_dict[most_negative_feature]

print(most_negative_feature, most_negative_weight)


{'carat': 5338.6156706147885, 'depth': -90.1738170334196, 'table': -60.33227991027331, 'x': -1100.4188500393068, 'y': -6.45891672053097, 'z': -34.2589446208815, 'cut_Good': 595.5210801260575, 'cut_Ideal': 846.1962575490013, 'cut_Premium': 777.7699627551737, 'cut_Very Good': 743.3013602895841, 'color_E': -217.0289677373036, 'color_F': -273.30397584051707, 'color_G': -500.1602381768932, 'color_H': -998.6346691711666, 'color_I': -1476.0842333201992, 'color_J': -2381.989788469752, 'clarity_IF': 5459.037299756213, 'clarity_SI1': 3751.7096712587204, 'clarity_SI2': 2781.9405481388126, 'clarity_VS1': 4657.834307077786, 'clarity_VS2': 4336.72558297791, 'clarity_VVS1': 5101.335181071201, 'clarity_VVS2': 5028.295530436609}
color_J -2381.989788469752


## Попытка улучшить качество модели

Данная часть является творческой, и задания в ней немного сложнее, чем предыдущие.

Следующие вопросы не проверяются тестами.

### Задание 9

Как можно заметить из анализа корреляционной матрицы в задании 3, между некоторыми признаками имеется сильная корреляция, что может быть индикатором проблемы *мультиколлинеарности*. Различия в порядке коэффициентов, выявленные в предыдущей задаче, также свидетельствуют об этом. Для решения этой проблемы можно либо исключить некоторые признаки из модели (например, если признак линейно зависим с какими-то другими, его можно исключить из модели, т.е. удалить из матрицы объект-признак и заново обучить модель).

Удалите из матриц `Xtrain` и `Xtest` признак, который наиболее сильно коррелирует с остальными. Заново обучите модель и оцените её качество. Улучшилось ли качество модели?

Попробуйте удалить какой-то другой признак (можете попробовать несколько вариантов). Помогло ли это улучшить качество модели?

In [ ]:
# your code here

### Задание 10

Иногда генерация новых признаков помогает модели лучше находить взаимосвязи между целевой переменной и признаками. Попробуйте придумать новые признаки и добавить их в модель.  

Помогло ли это улучшить качество модели?

In [ ]:
# your code here